In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Loading the Datasets 

In [3]:
dataset = pd.read_csv(r"C:\Users\ADMIN\OneDrive\Documents\Datasets\Bank+Customer+Churn\Bank_Churn_Data_Dictionary.csv")
dataset2 = pd.read_excel(r"C:\Users\ADMIN\OneDrive\Documents\Datasets\Bank+Customer+Churn\Bank_Churn_Messy.xlsx", sheet_name=None)
dataset3 = pd.read_csv(r"C:\Users\ADMIN\OneDrive\Documents\Datasets\Bank+Customer+Churn\Bank_Churn.csv")

In [4]:
# Extracting the sheets from the excel file

Customer_info = dataset2['Customer_Info']
Account_info = dataset2['Account_Info']

# Merging them into one DataFrame
merged_df = pd.merge(Customer_info, Account_info, on = 'CustomerId', how = 'inner')


## Data Cleaning


In [5]:
# Dropping unnecessary columns
merged_df.drop(columns = ['Tenure_x'], inplace = True)
merged_df.rename(columns = {'Tenure_y': 'Tenure'}, inplace = True)

In [6]:
merged_df.duplicated() # checking for duplicates
merged_df[merged_df.duplicated(keep=False)] 

,CustomerId,Surname,CreditScore,Geography,Gender,Age,EstimatedSalary,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,Hargrave,619,FRA,Female,42.0,€101348.88,€0.0,1,Yes,2,Yes,1
1,15634602,Hargrave,619,FRA,Female,42.0,€101348.88,€0.0,1,Yes,2,Yes,1
10000,15628319,Walker,792,French,Female,28.0,€38190.78,€130142.79,1,No,4,No,0
10001,15628319,Walker,792,French,Female,28.0,€38190.78,€130142.79,1,No,4,No,0
10002,15628319,Walker,792,French,Female,28.0,€38190.78,€130142.79,1,No,4,No,0
10003,15628319,Walker,792,French,Female,28.0,€38190.78,€130142.79,1,No,4,No,0


In [7]:
merged_df.drop_duplicates(inplace = True) # dropping duplicates

In [8]:
merged_df.reset_index(drop=True, inplace=True) # resetting the index

In [9]:
# Getting the first three indices of NaN values in the 'Surname' column
nan_indices = merged_df[merged_df['Surname'].isna()].index[:3]

replacements = ['McWilliams', 'Lombardo', 'Kennedy']

# Replacing the NaN values with the replacements
merged_df.loc[nan_indices, 'Surname'] = replacements

In [10]:
# Getting the first three indices of NaN values in the 'Age' column
nan_indices = merged_df[merged_df['Age'].isna()].index[:3]

# Creating a list of replacements for the NaN values
replacements = [43,39,32]

# Replacing the NaN values with the replacements
merged_df.loc[nan_indices, 'Age'] = replacements

In [11]:
merged_df['Surname'] = merged_df['Surname'].str.strip() # stripping the Surname column

In [12]:
# replacing the values in the Geography column
merged_df['Geography'] = merged_df['Geography'].replace({'FRA': 'France','French': 'France'})  

In [13]:
merged_df['Age'] = merged_df['Age'].astype(int) # converting the Age column to int

In [14]:
# replacing the €
merged_df['EstimatedSalary'] = merged_df['EstimatedSalary'].str.replace('€', '')
merged_df['Balance'] = merged_df['Balance'].str.replace('€', '') 

In [15]:
merged_df['Balance'] = merged_df['Balance'].astype(float) # converting the Balance column to float
merged_df['EstimatedSalary'] = merged_df['EstimatedSalary'].astype(float) # converting the EstimatedSalary column to float

In [16]:
# converting the HasCrCard column to int
merged_df['HasCrCard'] = merged_df['HasCrCard'].map({'Yes' : 1, 'No': 0}).astype(int)  

In [17]:
# converting the IsActiveMember column to int
merged_df['IsActiveMember'] = merged_df['IsActiveMember'].map({'Yes' : 1, 'No': 0}).astype(int)  

In [18]:
# Cleaned Data
merged_df.head(6)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,EstimatedSalary,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,Hargrave,619,France,Female,42,101348.88,0.00,1,1,2,1,1
1,15647311,Hill,608,Spain,Female,41,112542.58,83807.86,1,1,1,1,0
2,15619304,Onio,502,France,Female,42,113931.57,159660.80,3,0,8,0,1
3,15701354,Boni,699,France,Female,39,93826.63,0.00,2,0,1,0,0
4,15737888,Mitchell,850,Spain,Female,43,79084.10,125510.82,1,1,2,1,0
5,15574012,Chu,645,Spain,Male,44,149756.71,113755.78,2,0,8,0,1


# Exploratory Data Analysis

In [23]:
merged_df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'EstimatedSalary', 'Balance', 'NumOfProducts', 'HasCrCard', 'Tenure',
       'IsActiveMember', 'Exited'],
      dtype='object')

In [ ]:
# Percentage of churn by gender where 1 = exited and 0 = not exited
pd.crosstab(merged_df['Gender'], merged_df['Exited'], margins=True, normalize= 'index') * 100

Exited,0,1
Gender,,
Female,74.928461,25.071539
Male,83.544072,16.455928
All,79.630000,20.370000


In [ ]:
# Percentage of churn by geography where 1 = not exited and 0 = exited
pd.crosstab(merged_df['Geography'], merged_df['Exited'], margins=True, normalize= 'index') * 100

Exited,0,1
Geography,,
France,83.845233,16.154767
Germany,67.556796,32.443204
Spain,83.326605,16.673395
All,79.630000,20.370000
